In [1]:
import os
os.chdir('../')

In [3]:
%pwd

'e:\\EndToEnd\\kidney-disease-classification'

## Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    dataset_path: Path
    epochs: int
    batch_size: int
    augmentation: bool
    image_size: int

## Config

In [6]:
import tensorflow as tf
from kidneyDiseaseClassifire.utils.common import read_yaml, create_dirs
from kidneyDiseaseClassifire.constants import *

class ConfigManger:
    def __init__(self, 
                 config_path=CONFIG_FILEPATH,
                 params_path=PARAMS_FILEPATH,
                 ):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

    def get_training_config(self) -> ModelTrainingConfig:
        base_model_config = self.config.base_model
        trainig_config = self.config.model_tarining
        params = self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_data_path, "kidney-images")
        create_dirs([trainig_config.root_dir])
        
        return ModelTrainingConfig(
            root_dir= trainig_config.root_dir,
            trained_model_path= trainig_config.tarined_model_path,
            updated_model_path= base_model_config.updated_model_path,
            dataset_path= Path(training_data),
            epochs= params.EPOCHS,
            batch_size= params.BATCH_SIZE,
            augmentation= params.AUGMENTATION,
            image_size= params.IMAGE_SIZE
        )

[2024-01-24 00:35:15,037: INFO: utils: NumExpr defaulting to 4 threads.]


## Components

In [ ]:
import os
import tensorflow as tf
import urllib.request as request
import time


class ModelTraining:
    def __init__(self,
                 config: ModelTrainingConfig,
                 ):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_model_path)